# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
import hvplot.pandas
import pandas as pd
import requests
import folium

from api_keys import geoapify_key

In [2]:
file_path = "city_weather_data.csv"

city_data_df = pd.read_csv(file_path)

display(city_data_df.head())


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.6001,-73.9662,282.49,41,1,1.79,US,1709326824
1,afaahiti,-17.7500,-149.2833,302.08,78,54,7.13,PF,1709326874
2,badger,64.8000,-147.5333,247.79,70,20,0.00,US,1709326875
3,san antonio de pale,-1.4014,5.6325,301.90,77,77,2.97,GQ,1709326875
4,pariaman,-0.6190,100.1200,298.18,85,100,0.69,ID,1709326876


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
m = folium.Map(location=[0, 0], zoom_start=2)

for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 5,  # Adjust the divisor to control the size of the markers
        popup=row['City'],
        color='red',
        fill=True,
        fill_color='pink'
    ).add_to(m)

folium.TileLayer('Stamen Terrain').add_to(m)

m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
filtered_cities_df = city_data_df[((city_data_df['Max Temp'] >= 294.15) & 
                                   (city_data_df['Max Temp'] <= 300.15)) & 
                                  (city_data_df['Wind Speed'] < 4.5) & 
                                  (city_data_df['Cloudiness'] == 0)]

filtered_cities_df = filtered_cities_df.dropna()

display(filtered_cities_df.head())


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,anahuac,27.2333,-100.1500,297.23,24,0,1.86,MX,1709326897
206,vail,32.0479,-110.7120,296.45,21,0,1.34,US,1709326985
244,howrah,22.5892,88.3103,294.15,68,0,0.00,IN,1709326994
293,rajula,21.0500,71.4333,296.77,70,0,3.93,IN,1709326802
448,richards bay,-28.7830,32.0377,296.26,81,0,4.09,ZA,1709327043


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

hotel_df['Hotel Name'] = ""

hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
40,anahuac,MX,27.2333,-100.1500,24,
206,vail,US,32.0479,-110.7120,21,
244,howrah,IN,22.5892,88.3103,68,
293,rajula,IN,21.0500,71.4333,70,
448,richards bay,ZA,-28.7830,32.0377,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000000 
params = {
    "categories": "accommodation.hotel", 
    # "radius": radius,
    "apiKey": geoapify_key,
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    base_url = "https://api.geoapify.com/v2/places"
    
    response = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df.head()


Starting hotel search
anahuac - nearest hotel: No hotel found
vail - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
rajula - nearest hotel: Hotel Akshardham
richards bay - nearest hotel: SUN1 Richards Bay
blackwater - nearest hotel: No hotel found
beitbridge - nearest hotel: Beitbridge Inn & Casino
uttar char fasson - nearest hotel: No hotel found
tlahualilo de zaragoza - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
40,anahuac,MX,27.2333,-100.1500,24,No hotel found
206,vail,US,32.0479,-110.7120,21,No hotel found
244,howrah,IN,22.5892,88.3103,68,No hotel found
293,rajula,IN,21.0500,71.4333,70,Hotel Akshardham
448,richards bay,ZA,-28.7830,32.0377,81,SUN1 Richards Bay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
map_center = [hotel_df.iloc[0]['Lat'], hotel_df.iloc[0]['Lng']]
map = folium.Map(location=map_center, zoom_start=10)

for index, row in hotel_df.iterrows():
    folium.Marker(location=[row['Lat'], row['Lng']], popup=row['City']).add_to(map)

map
